# Install and load necesary packages

In [1]:
# Please don't change this cell

import pandas as pd
import numpy as np  

import warnings
warnings.filterwarnings("ignore")

## Load the dataset using pandas

In [2]:
# Please don't change this cell
df = pd.read_csv('ml-100k/u.data', names=['user_id', 'item_id', 'rating', 'timestamp'], sep='\t')

df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# Split dataset


## Randomly select one rating from each user as test set

In [3]:
# please do not change this cell

from sklearn.model_selection import train_test_split

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print(str(n_users) + ' users')
print(str(n_items) + ' items')

train_df, test_df = train_test_split(df, test_size=0.2, random_state = 10)
train_df, test_df

# Training Dataset
train_ds = np.zeros((n_users, n_items))
item_popularity = np.zeros(n_items)
for row in train_df.itertuples():
    train_ds[row[1]-1, row[2]-1] = row[3]
    item_popularity[row[2]-1] =  item_popularity[row[2]-1] + 1
#train_ds = pd.DataFrame(train_ds)

# Testing Dataset
testsize = 0
test_ds = np.zeros((n_users, n_items))
for row in test_df.itertuples():
    if item_popularity[row[2]-1] > 30:
        test_ds[row[1]-1, row[2]-1] = row[3]
        testsize = testsize + 1
#test_ds = pd.DataFrame(test_ds)

print("Construct the rating matrix based on train_df:")
print(train_ds)

print("Construct the rating matrix based on test_df:")
print(test_ds)

print("Testsize = " + str(testsize))

943 users
1682 items
Construct the rating matrix based on train_df:
[[0. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
Construct the rating matrix based on test_df:
[[5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Testsize = 17678


# MAE and RMSE Utils

In [4]:
# Please don't change this cell
# you can use this devaluate Utils here, and you can also implement your own MAE and RMSE calculation. 

EPSILON = 1e-9

def evaluate(test_ds, predicted_ds):
    '''
    Function for evaluating on MAE and RMSE
    '''
    # MAE
    mask_test_ds = test_ds > 0
    MAE = np.sum(np.abs(test_ds[mask_test_ds] - predicted_ds[mask_test_ds])) / np.sum(mask_test_ds.astype(np.float32))

    # RMSE
    RMSE = np.sqrt(np.sum(np.square(test_ds[mask_test_ds] - predicted_ds[mask_test_ds])) / np.sum(mask_test_ds.astype(np.float32)))

    return MAE, RMSE

In [5]:
train_ds.shape

(943, 1682)

# Your Solution for Method 1

In [6]:
# Write your code here for Method 1
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

#Calculate average rating for each user
user_averages = []
for i in range(train_ds.shape[0]):  # Loop through users
    ratings = train_ds[i]
    rated = ratings[ratings > 0]  # Consider only non-zero ratings
    
    if rated.size > 0:
        avg = np.mean(rated)
    else:
        avg = 0  # You can also use np.nan or global average if preferred
    
    user_averages.append(avg)

user_averages = np.array(user_averages)

# Predict ratings using user averages 
# Following the rule: ˆra,i = ¯ua (user a's average)

predicted_ds = []
for i in range(test_ds.shape[0]):  # Loop through users
    row = []
    for j in range(test_ds.shape[1]):  # Loop through items
        if test_ds[i][j] > 0:  # Only predict for actual test ratings
            row.append(user_averages[i])
        else:
            row.append(0)  # Leave unrated items as 0
    predicted_ds.append(row)

predicted_ds = np.array(predicted_ds)


MAE_solution1, RMSE_solution1 = evaluate(test_ds, predicted_ds)

print(f"MAE: {MAE_solution1:.4f}")
print(f"RMSE: {RMSE_solution1:.4f}")

        

MAE_solution1 = 0.8258905090105331 # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.
RMSE_solution1 = 1.031143070595138 # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.



MAE: 0.8259
RMSE: 1.0311


In [7]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution1, RMSE_solution1))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.8258905090105331, RMSE: 1.031143070595138


# Your Solution for Method 2

In [8]:
# Write your code here for Method 2
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

# Item Average
# Ratings are numbers or 0 if not rated

# Calculate average rating for each item
item_averages = []
for i in range(train_ds.shape[1]):  # Loop through items (columns)
    ratings = train_ds[:, i]  # Get all ratings for item i
    rated = ratings[ratings > 0]  # Consider only non-zero ratings
    
    if rated.size > 0:
        avg = np.mean(rated)
    else:
        avg = 0  # You can also use np.nan or global average if preferred
    
    item_averages.append(avg)
item_averages = np.array(item_averages)

# Predict ratings using item averages 
# Following the rule: ˆra,i = t̄i (item i's average)
predicted_ds = []
for i in range(test_ds.shape[0]):  # Loop through users
    row = []
    for j in range(test_ds.shape[1]):  # Loop through items
        if test_ds[i][j] > 0:  # Only predict for actual test ratings
            row.append(item_averages[j])  # Use item j's average rating
        else:
            row.append(0)  # Leave unrated items as 0
    predicted_ds.append(row)
predicted_ds = np.array(predicted_ds)

MAE_solution2, RMSE_solution2 = evaluate(test_ds, predicted_ds)
print(f"MAE: {MAE_solution2:.4f}")
print(f"RMSE: {RMSE_solution2:.4f}")
    



# Average rating of movie and model trainig and testing 
MAE_solution2 = 0.7961 # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.
RMSE_solution2 = 1.0013 # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.


MAE: 0.7961
RMSE: 1.0013


In [9]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution2, RMSE_solution2))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.7961, RMSE: 1.0013


# Your Solution for Method 3

In [10]:
# Write your code here for Method 3
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

# User KNN based 
# Calculate User-User Similarity Matrix 
Delta = 25
EPSILON = 1e-9
K = 5  # Number of nearest neighbors to consider

# Initialize user-user similarity matrix
np_user_pearson_corr = np.zeros((n_users, n_users))

print("Calculating user-user similarities...")
for i in range(n_users):
    for j in range(n_users):
        if i == j:
            np_user_pearson_corr[i, j] = 1.0  # Perfect similarity with self
            continue
        
        # Find items rated by both users
        mask_i = train_ds[i] > 0
        mask_j = train_ds[j] > 0
        common_items = np.where(mask_i & mask_j)[0]
        
        if len(common_items) == 0:
            continue  # No common items rated
        
        # Get ratings for common items
        user_i_ratings = train_ds[i, common_items]
        user_j_ratings = train_ds[j, common_items]
        
        # Calculate Pearson correlation
        mean_i = np.mean(user_i_ratings)
        mean_j = np.mean(user_j_ratings)
        
        numerator = np.sum((user_i_ratings - mean_i) * (user_j_ratings - mean_j))
        denominator = np.sqrt(np.sum((user_i_ratings - mean_i)**2)) * np.sqrt(np.sum((user_j_ratings - mean_j)**2))
        
        if denominator == 0:
            sim = 0
        else:
            sim = numerator / (denominator + EPSILON)
        
        # Apply significance weighting
        weighted_sim = (min(len(common_items), Delta) / Delta) * sim
        np_user_pearson_corr[i, j] = weighted_sim

# Predict Ratings for Test Set
print("Making predictions...")

# Initialize prediction matrix
predicted_ds = np.zeros_like(test_ds)

# Pre-calculate user averages for fallback
user_averages = np.sum(train_ds, axis=1) / (np.sum(train_ds > 0, axis=1) + EPSILON)

for user_idx in range(test_ds.shape[0]):
    for item_idx in range(test_ds.shape[1]):
        # Only predict for ratings that exist in test set
        if test_ds[user_idx, item_idx] > 0:
            
            # Find K most similar users
            similarities = np_user_pearson_corr[user_idx]
            neighbor_indices = np.argsort(similarities)[-(K+1):-1][::-1]  # Top K, excluding self
            
            # Weighted sum of ratings from neighbor users
            numerator = 0
            denominator = 0
            
            for neighbor_user_idx in neighbor_indices:
                # Check if neighbor user rated this item
                neighbor_rating = train_ds[neighbor_user_idx, item_idx]
                if neighbor_rating > 0:
                    neighbor_similarity = np_user_pearson_corr[user_idx, neighbor_user_idx]
                    
                    # Use absolute similarity for weighting
                    denominator += abs(neighbor_similarity)
                    numerator += neighbor_similarity * neighbor_rating
            
            # Make prediction
            if denominator > 0:
                prediction = numerator / denominator
            else:
                # Fallback: Use user's average rating
                prediction = user_averages[user_idx]
            
            predicted_ds[user_idx, item_idx] = prediction

# Evaluate Predictions
# Clip predictions to valid range [1, 5]
predicted_ds = np.clip(predicted_ds, 1, 5)

# Evaluate using provided function
MAE_solution3, RMSE_solution3 = evaluate(test_ds, predicted_ds)

print(f"Method 3 (User KNN) Results:")
print(f"MAE: {MAE_solution3:.4f}")
print(f"RMSE: {RMSE_solution3:.4f}")
print(f"K value used: {K}")

# Store best user prediction for Method 5
best_user_prediction = predicted_ds.copy()


MAE_solution3 = 0.8737 # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.
RMSE_solution3 = 1.1399 # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.


Calculating user-user similarities...
Making predictions...
Method 3 (User KNN) Results:
MAE: 0.8737
RMSE: 1.1399
K value used: 5


In [11]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution3, RMSE_solution3))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.8737, RMSE: 1.1399


# Your Solution for Method 4

In [12]:
# Write your code here for Method 4
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

# Item KNN based
Delta = 25 
EPSILON = 1e-9
K = 5

# Similarity Matrix
np_item_pearson_corr = np.zeros((n_items, n_items))  # Pearson Correlation 

# Now lets transpose the training matrix to iterate over items
train_items_ds = train_ds.T  # Reshaping
print("Calculating item similarities...")
# Loop for similarity calculation
for i, item_i_ratings in enumerate(train_items_ds):  # Enumerate returns data and rows
    for j, item_j_ratings in enumerate(train_items_ds):  # Creates possible item pairs
        if i == j:
            np_item_pearson_corr[i, j] = 1.0  # Highly likely
            continue
        
        # users who have rated both items
        mask_i = item_i_ratings > 0 
        mask_j = item_j_ratings > 0   # Mask for item i, j 
        
        corrated_user_indices = np.where(mask_i & mask_j)[0]  # Finds users who rated both I and J
        if len(corrated_user_indices) == 0:
            continue    # Skips if no users rated that item 
            
        # Extract Co-rated Values
        item_i_corated = item_i_ratings[corrated_user_indices]
        item_j_corated = item_j_ratings[corrated_user_indices]
        
        # pearson calculation
        mean_i = np.mean(item_i_corated)  # Average of i value 
        mean_j = np.mean(item_j_corated)  # Average of j VALUE
        
        numerator = np.sum((item_i_corated - mean_i) * (item_j_corated - mean_j))
        denominator = np.sqrt(np.sum((item_i_corated - mean_i)**2)) * np.sqrt(np.sum((item_j_corated - mean_j)**2))
        
        # Gives us the output from 1 to -1 range close to 1 highly Likely close to -1 Dislike
        
        # main calculation code
        if denominator == 0:
            sim = 0
        else:
            sim = numerator / (denominator + EPSILON)
            
        # Weighting
        weighted_sim = (min(len(corrated_user_indices), Delta) / Delta) * sim
        np_item_pearson_corr[i, j] = weighted_sim  # Store the calculated weighted similarity in the matrix

# Prediction
# Now lets predict Ratings
predicted_ds = np.zeros_like(test_ds)

item_averages = np.sum(train_ds, axis=0) / (np.sum(train_ds > 0, axis=0) + EPSILON)

# User prediction Loop
for user_idx in range(test_ds.shape[0]):
    for item_idx in range(test_ds.shape[1]):
        if test_ds[user_idx, item_idx] > 0:  # predict ratings inside the test set
            
            similarities = np_item_pearson_corr[item_idx]  # Get Similarity scores
            neighbor_indices = np.argsort(similarities)[-(K+1):-1][::-1]
            
            # Weighted sum of rating from neighbors
            numerator = 0
            denominator = 0
            
            for neighbor_item_idx in neighbor_indices:
                user_rating_for_neighbor = train_ds[user_idx, neighbor_item_idx]  # Users rating for the similar item from training data
                
                if user_rating_for_neighbor > 0:
                    neighbor_similarity = np_item_pearson_corr[item_idx, neighbor_item_idx]
                    denominator += abs(neighbor_similarity)  # abs = absolute value
                    numerator += neighbor_similarity * user_rating_for_neighbor  # Accumulate Weighted ratings
            
            # Prediction
            if denominator > 0:
                prediction = numerator / denominator
            else:
                prediction = item_averages[item_idx]
                
            predicted_ds[user_idx, item_idx] = prediction
# MAE AND RMSE
predicted_ds = np.clip(predicted_ds, 1, 5) 
MAE_solution4, RMSE_solution4 = evaluate(test_ds, predicted_ds)

# Store for 5TH
best_item_prediction = predicted_ds.copy()
print(f"K value used: {K}")
print(f"MAE from running{MAE_solution4}")
print(f"RMSE from running{RMSE_solution4}")
    
MAE_solution4 = 0.8574358073342582 # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.
RMSE_solution4 = 1.1357385666743622 # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.


Calculating item similarities...
K value used: 5
MAE from running0.8574358073342582
RMSE from running1.1357385666743622


In [13]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution4, RMSE_solution4))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.8574358073342582, RMSE: 1.1357385666743622


# Your Solution for Method 5

In [14]:
# Write your code here for Method 5
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

# Method 5: Hybrid Collaborative Filtering


lambda_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
best_mae = float('inf')
best_lambda = 0

print("Method 5: Testing different lambda values")
print("-" * 40)

for lambda_val in lambda_values:
    # Combine user and item predictions
    combined_result = lambda_val * best_user_prediction + (1 - lambda_val) * best_item_prediction
    combined_result = np.clip(combined_result, 1, 5)
    
    # Evaluate
    MAE_solution5, RMSE_solution5 = evaluate(test_ds, combined_result)
    print(f"Lambda = {lambda_val:.1f}: MAE = {MAE_solution5:.4f}, RMSE = {RMSE_solution5:.4f}")
    
    # Keep track of best performance
    if MAE_solution5 < best_mae:
        best_mae = MAE_solution5
        best_rmse = RMSE_solution5
        best_lambda = lambda_val
        best_predictions = combined_result.copy()

print("-" * 40)
print(f"Best: Lambda = {best_lambda}, MAE = {best_mae:.4f}, RMSE = {best_rmse:.4f}")


# User item KNN based
MAE_solution5 = best_mae # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.
RMSE_solution5 = best_rmse # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.


Method 5: Testing different lambda values
----------------------------------------
Lambda = 0.1: MAE = 0.8420, RMSE = 1.0999
Lambda = 0.2: MAE = 0.8288, RMSE = 1.0712
Lambda = 0.3: MAE = 0.8186, RMSE = 1.0504
Lambda = 0.4: MAE = 0.8128, RMSE = 1.0379
Lambda = 0.5: MAE = 0.8111, RMSE = 1.0340
Lambda = 0.6: MAE = 0.8164, RMSE = 1.0388
Lambda = 0.7: MAE = 0.8257, RMSE = 1.0522
Lambda = 0.8: MAE = 0.8390, RMSE = 1.0739
Lambda = 0.9: MAE = 0.8553, RMSE = 1.1033
----------------------------------------
Best: Lambda = 0.5, MAE = 0.8111, RMSE = 1.0340


In [15]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution5, RMSE_solution5))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.8111018800910201, RMSE: 1.0340053171084422
